In [1]:
# Jozef Porubcin
# Semester Project
# Code adapted from Practical 2: Skeleton credit to Yifan Qin, Zheng Ning, and Adam Czajka

In [2]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision.transforms.functional import resize
from torchvision.transforms import CenterCrop
from torchvision.transforms import ToTensor
from torchvision.io import read_image
from torchsummary import summary
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import csv
import random

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
class CNN(nn.Module):
    def __init__(self, numChannels, numClasses):
        super(CNN, self).__init__()
        self.classes = numClasses
        
        conv_out1 = 96
        conv_out2 = 256
        conv_out3 = 384
        conv_out4 = 384
        conv_out5 = 256
        self.conv1 = torch.nn.Conv2d(numChannels, conv_out1, (11, 11), (4, 4))
        self.conv2 = torch.nn.Conv2d(conv_out1, conv_out2, (5, 5), (1, 1))
        self.conv3 = torch.nn.Conv2d(conv_out2, conv_out3, (3, 3), (1, 1))
        self.conv4 = torch.nn.Conv2d(conv_out3, conv_out4, (3, 3), (1, 1))
        self.conv5 = torch.nn.Conv2d(conv_out4, conv_out5, (3, 3), (1, 1))

        self.relu = torch.nn.ReLU()

        self.maxpool = torch.nn.MaxPool2d((2, 2), (2, 2))

        self.batchnorm1 = torch.nn.BatchNorm2d(conv_out1)
        self.batchnorm2 = torch.nn.BatchNorm2d(conv_out2)

        fc1_out1 = 1024
        self.fc1 = torch.nn.Linear(conv_out5*15, fc1_out1)
        self.fc2 = torch.nn.Linear(fc1_out1, numClasses)

    def evaluate(self, model, dataloader, classes, device):

        # We need to switch the model into the evaluation mode
        model.eval()
        
        correct_pred = {classname: 0 for classname in classes}
        total_pred = {classname: 0 for classname in classes}

        for data in dataloader:
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predictions = torch.max(outputs, 1)

            images = images.detach().cpu().numpy()
            labels = labels.detach().cpu().numpy()
            predictions = predictions.detach().cpu().numpy()

            for label, prediction in zip(labels, predictions):
                
                # If you want to see real and predicted labels for all samples:
                print("Real class: " + classes[label] + ", predicted = " + classes[prediction])
                
                if label == prediction:
                    correct_pred[classes[label]] += 1
                total_pred[classes[label]] += 1

        acc = sum(correct_pred.values()) / sum(total_pred.values())

        return acc


    def forward(self, x):
        x = resize(x, size=[256], antialias=True)

        # First convolutional block
        # self.conv1 -> self.relu -> self.maxpool -> self.batchnorm1
        x = self.conv1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.batchnorm1(x)

        # Secondconvolutional block
        # self.conv2 -> self.relu -> self.maxpool -> self.batchnorm2
        x = self.conv2(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.batchnorm2(x)

        # Third convolutional block
        # self.conv3 -> self.relu
        x = self.conv3(x)
        x = self.relu(x)

        # Fourth convolutional block
        # self.conv4 -> self.relu
        x = self.conv4(x)
        x = self.relu(x)

        # Fifth convolutional block
        # self.conv5 -> self.relu -> self.maxpool
        x = self.conv5(x)
        x = self.relu(x)
        x = self.maxpool(x)


        # After the last pooling operation, and before the first 
        # fully-connected layer, we need to "flatten" our tensors
        x = torch.flatten(x, 1)


        # self.fc1 -> self.relu -> self.fc2
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)

        return x

In [5]:
# Check which layers are "trainable"
def check(model):
    print("Layers to be fine-tuned:")
    for name, param in model.named_parameters():
        if param.requires_grad == True:
            print(name)


In [6]:
class FilteredFruits(Dataset):
    def __init__(self, img_dir, label_dir):
        self.img_dir = img_dir
        self.img_labels = pd.read_csv(label_dir, header=0)
        self.noise = 0
        self.random_chance = 0.0

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = read_image(img_path)
        image = image.to(torch.float) / 256.
        label = self.img_labels.iloc[idx, 1]
        if self.noise == 1:
            im = image.numpy()
            
            for row in range(im.shape[1]):
                for col in range(im.shape[2]):
                    num = random.random()
                    if(num < self.random_chance):
                        R = random.random()
                        G = random.random()
                        B = random.random()
                        im[0, row, col] = R
                        im[1, row, col] = G
                        im[2, row, col] = B
            
            image = im

        return (image, label)

In [7]:
def generate_labels(image_dir, label_dir):
    row_list = [['file', 'label']]

    min_images = float('inf')
    for class_dir in os.scandir(image_dir):
        if class_dir.is_dir():
            min_images = min(len(os.listdir(class_dir)), min_images)

    l = 0
    for class_dir in os.scandir(image_dir):
        if class_dir.is_dir():
            img_files = os.listdir(class_dir)
            random.shuffle(img_files)
            for x in range(min_images):
                row_list.append([(class_dir.name + '/' + img_files[x]), l])

            l += 1
    
    with open(label_dir, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerows(row_list)

In [8]:
def generate_classes():
    f = open("/content/drive/MyDrive/Semester_Project/fruits/fruits_labels.csv", "r")
    data = list(csv.reader(f, delimiter=","))
    f.close()
    data = data[1:]

    classes = []
    i = -1
    for pair in data:
        if i < int(pair[1]):
            i = int(pair[1])
            classes.append(pair[0].split('/')[0])

    return classes

In [9]:
if __name__ == '__main__':
    # Path to the fruits dataset:
    image_dir = '/content/drive/MyDrive/Semester_Project/fruits/'

    # Path to the labels of the fruits subset
    label_dir = '/content/drive/MyDrive/Semester_Project/fruits/fruits_labels.csv'

    generate_labels(image_dir, label_dir)

    transfer_learning = 'all_layers'

    epochs = 200
    batch_size = 32

    # Load sampled fruits dataset
    data = FilteredFruits(img_dir=image_dir, label_dir=label_dir)
    
    train_len = int(len(data) * 0.8)
    val_len = int(len(data) * 0.1)
    test_len = int(len(data) - train_len - val_len)
    print(len(data))
    train_data, val_data, test_data = random_split(data, [train_len, val_len, test_len])

    print(len(train_data))
    print(len(val_data))
    print(len(test_data))

    single_test_len = 1
    rest_test_len = test_len - 1
    single_test_data, _ = random_split(test_data, [1, test_len - 1])

    classes = generate_classes()

    # Prepare data loaders for train, validation and test data splits
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=False, drop_last=False, num_workers=2)
    val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False, drop_last=False, num_workers=2)
    test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, drop_last=False, num_workers=2)

    #######################################################
    # Specify the operation mode:
    # 'train' = training with your train and validation data splits
    # 'eval'  = evaluation of the trained model with your test data split
    mode = 'special'

    # Path where you will save the best model after fine-tuning
    my_best_model = "/content/drive/MyDrive/Semester_Project/best_model.pth"

    #
    #######################################################


    # Set the device (GPU or CPU, depending on availability)
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    print("Currently using device: ", device)

    # Initialize the model
    model_t = CNN(numChannels = 3, numClasses = len(classes))
    model_t.to(device)
    
    # Training happens here!
    if mode == "train":

        print("\n\nFine-tuning (transfer learning) starts!\n\n")

        model_t.train()
        criterion = nn.CrossEntropyLoss()
        #optimizer = torch.optim.Adam(model_t.parameters(), lr=0.001)
        optimizer = torch.optim.SGD(model_t.parameters(), lr=0.001, momentum=0.9)
        
        my_starting_weights = my_best_model

        if my_starting_weights and transfer_learning != "none":
            try:
                print(f"Loading the weights from {my_starting_weights} ...")
                model_t.load_state_dict(torch.load(my_starting_weights))
                print("Successfully loaded the model checkpoint!")
            except FileNotFoundError:
                print(f"Model checkpoint {my_starting_weights} does not exist.")
        
        # Sanity check: print layers that are being fine-tuned (not frozen)
        print(check(model_t))

        running_loss = .0
        best_acc = .0
        for epoch in range(epochs):
            print(f"Starting epoch {epoch + 1}")
            for idx, data in tqdm(enumerate(train_loader), total=len(train_loader)):
                # Get the inputs (data is a list of [inputs, labels])
                inputs, labels = data
                '''
                if(idx == 0):
                    im = inputs.numpy()
                    im = np.einsum('ijkl->iklj', im)
                    plt.figure(figsize=(8, 4))
                    for i in range(im.shape[0]):
                        plt.subplot(4, 8, i+1)
                        #plt.axis('off')
                        plt.imshow(im[i])
                    plt.show()
                '''
                inputs = inputs.to(device)
                labels = labels.to(device)
                optimizer.zero_grad()
                outputs = model_t(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

                loss = loss.detach().cpu().numpy()
                inputs = inputs.detach().cpu().numpy()
                labels = labels.detach().cpu().numpy()
                running_loss += loss

            # Evaluate the accuracy after each epoch
            acc = model_t.evaluate(model_t, val_loader, classes, device)
            if acc > best_acc:
                print(f"Better validation accuracy achieved: {acc * 100:.2f}%")
                best_acc = acc
                print(f"Saving this model as: {my_best_model}")
                torch.save(model_t.state_dict(), my_best_model)

    # And here we evaluate the trained model with the test data
    elif mode == "eval":
        test_loader.dataset.dataset.noise = 1
        print(f"Loading checkpoint from {my_best_model}")
        model_t.load_state_dict(torch.load(my_best_model))
        acc = model_t.evaluate(model_t, test_loader, classes, device)
        print(f"Accuracy on the test (unknown) data: {acc * 100:.2f}%")

    elif mode == "special":
        print(f"Loading checkpoint from {my_best_model}")
        model_t.load_state_dict(torch.load(my_best_model))
        single_test_data.dataset.dataset.noise = 1

        num_iters = 10
        for x in range(num_iters):
            print()
            single_test_data.dataset.dataset.random_chance = x / num_iters
            single_loader = DataLoader(single_test_data, batch_size=1, shuffle=False, drop_last=False, num_workers=2)

            for data in single_loader:
                inputs, labels = data

                im = inputs.numpy()
                im = np.einsum('ijkl->iklj', im)
                plt.figure(figsize=(1, 1))
                for i in range(im.shape[0]):
                    plt.subplot(1, 1, i+1)
                    plt.axis('off')
                    plt.imshow(im[i])
                plt.show()

            print(f"Random noise: {int(x / num_iters * 100)}%")
            acc = model_t.evaluate(model_t, single_loader, classes, device)
            print(f"Accuracy on the test (unknown) data: {acc * 100:.2f}%")

    else:
        print("'mode' argument should either be 'train' or 'eval'")

11494
9195
1149
1150
Currently using device:  cuda:0
Loading checkpoint from /content/drive/MyDrive/Semester_Project/best_model.pth
Real class: raspberry, predicted = raspberry
Real class: raspberry, predicted = raspberry
Real class: pear, predicted = pear
Real class: orange, predicted = orange
Real class: pineapple, predicted = pineapple
Real class: cherry, predicted = cherry
Real class: banana, predicted = banana
Real class: blackberry, predicted = blackberry
Real class: blackberry, predicted = blackberry
Real class: blueberry, predicted = blueberry
Real class: pomegranate, predicted = pomegranate
Real class: blueberry, predicted = blueberry
Real class: blueberry, predicted = blueberry
Real class: cherry, predicted = cherry
Real class: grape, predicted = grape
Real class: grape, predicted = grape
Real class: apple, predicted = apple
Real class: apple, predicted = apple
Real class: strawberry, predicted = strawberry
Real class: banana, predicted = banana
Real class: grape, predicted =